In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn

import torchvision.datasets as datasets

# Datasets

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomCrop(32),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomRotation(180),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 128

In [9]:
class CIFAR10_corrupted(datasets.CIFAR10):
  def __init__(self, corrupt_prob=0.0, num_classes=10, **kwargs):
    super(CIFAR10_corrupted, self).__init__(**kwargs)
    self.n_classes = num_classes
    if corrupt_prob > 0:
      self.corrupt_labels(corrupt_prob)

  def corrupt_labels(self, corrupt_prob):
    labels = np.array(self.targets)
    np.random.seed(12345)
    mask = np.random.rand(len(labels)) <= corrupt_prob
    rnd_labels = np.random.choice(self.n_classes, mask.sum())
    labels[mask] = rnd_labels
    # we need to explicitly cast the labels from npy.int64 to
    # builtin int type, otherwise pytorch will fail...
    labels = [int(x) for x in labels]

    self.targets = labels

#### CIFAR-10

In [10]:
trainset_10 = CIFAR10_corrupted(root='./data', train=True,
                                        download=True, transform=transform)
trainloader_10 = torch.utils.data.DataLoader(trainset_10, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valset_10 = CIFAR10_corrupted(root='./data', train=False,
                                       download=True, transform=transform)
valloader_10 = torch.utils.data.DataLoader(valset_10, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Models

MLP-L-H

In [11]:
class MLP(nn.Module):
  def __init__(self, L, H, fixdim, simplex):
    super().__init__()
    
    self.relu = nn.ReLU()

    layers = [nn.Linear(3072, H)]
    bns = [nn.BatchNorm1d(H)]

    num_classes = 10

    for i in range(L-2):
      layers.append(nn.Linear(H, H))
      bns.append(nn.BatchNorm1d(H))
        
    if fixdim:
      layers.append(nn.Linear(H, num_classes))
      self.fc_out= nn.Linear(num_classes, num_classes)
    else:
      layers.append(nn.Linear(H, H))
      self.fc_out = nn.Linear(H, num_classes)
    
    bns.append(nn.BatchNorm1d(H))

    self.fcs = nn.ModuleList(layers)
    self.bns = nn.ModuleList(bns)

    if simplex:
      weight = torch.sqrt(torch.tensor(num_classes/(num_classes-1)))*(torch.eye(num_classes)-(1/num_classes)*torch.ones((num_classes, num_classes)))
      weight /= torch.sqrt((1/num_classes*torch.norm(weight, 'fro')**2))
      if fixdim:
        self.fc_out.weight = nn.Parameter(weight)
      else:
        self.fc_out.weight = nn.Parameter(torch.mm(weight, torch.eye(num_classes, H)))
      self.fc_out.weight.requires_grad_(False)
    
  def forward(self, X):
    ret = {}
    h = X
    for i in range(len(self.fcs)):
      h = self.fcs[i](h)
      ret[str(i+1)] = h
      count += 1
      h = self.relu(self.bns[i](h))
    h = self.fc_out(h)
    ret['final'] = h
    return ret, len(self.fcs)

CONV-L-H

In [22]:
class CONV(nn.Module):
  def __init__(self, L, H, simplex):
    super().__init__()
    
    self.relu = nn.ReLU()

    starting_layers =  [nn.Conv2d(3, H, kernel_size=2, stride=2), nn.Conv2d(H, H, kernel_size=2, stride=2)]
    starting_bns = [nn.BatchNorm2d(H), nn.BatchNorm2d(H)]

    self.conv_in = nn.ModuleList(starting_layers)
    self.bn_in = nn.ModuleList(starting_bns)

    layers = []
    bns = []

    num_classes = 10

    for i in range(L):
      layers.append(nn.Conv2d(H, H, kernel_size=3, stride=1, padding=1))
      bns.append(nn.BatchNorm2d(H))
        
    self.fc_out = nn.Linear(64*H, num_classes)
    self.classify = nn.Linear(num_classes, num_classes)

    self.convs = nn.ModuleList(layers)
    self.bns = nn.ModuleList(bns)

    if simplex:
      weight = torch.sqrt(torch.tensor(num_classes/(num_classes-1)))*(torch.eye(num_classes)-(1/num_classes)*torch.ones((num_classes, num_classes)))
      weight /= torch.sqrt((1/num_classes*torch.norm(weight, 'fro')**2))
      self.classify.weight = nn.Parameter(weight)
      self.classify.weight.requires_grad_(False)
    
  def forward(self, X):
    h = X
    for layer, bn in zip(self.conv_in, self.bn_in):
      h = bn(layer(h))
    h = self.relu(h)

    for i in range(len(self.convs)):
      h = self.convs[i](h)
      h = self.relu(self.bns[i](h))
    h = h.reshape(X.shape[0], -1)
    h = self.fc_out(h)
    h = self.classify(h)
    return h, torch.argmax(h, dim=1)

  def get_layer_info(self, X):
    ret = {}
    h = X
    for layer, bn in zip(self.conv_in, self.bn_in):
      h = bn(layer(h))
    h = self.relu(h)
    for i in range(len(self.convs)):
      h = self.convs[i](h)
      ret[str(i+1)] = h
      h = self.relu(self.bns[i](h))
    h = h.reshape(X.shape[0], -1)
    h = self.fc_out(h)
    ret['final'] = h
    return ret

# Evaluations

In [51]:
from sklearn.neighbors import NearestCentroid
import numpy as np
import math

Evaluations

In [67]:
# NCC accuracy
def ncc_accuracy(features, classes):
  clf = NearestCentroid()
  clf.fit(features, classes)
  preds = np.array([clf.predict(features)]).T
  preds = preds.squeeze()
  assert preds.shape == classes.shape
  return np.sum(preds == classes) / preds.shape[0]

# class distance normalized variance:
# (Var(Q1) + Var(Q2)) / (2 ||mu(Q1) - mu(Q2)||^2)
# average over all pairs of nonequal classes
def var(Q):
  mu = np.mean(Q, axis=0)
  return np.mean(np.linalg.norm(Q - mu, axis=0)**2)

def cdnv(Q1, Q2):
  print('beep')
  print(var(Q1))
  assert not math.isnan(var(Q1))
  print(var(Q2))
  assert not math.isnan(var(Q2))
  print(2*np.linalg.norm(np.mean(Q1, axis=0) - np.mean(Q2, axis=0))**2)
  assert not math.isnan(2*np.linalg.norm(np.mean(Q1, axis=0) - np.mean(Q2, axis=0))**2)
  return (var(Q1) + var(Q2)) / (2*np.linalg.norm(np.mean(Q1, axis=0) - np.mean(Q2, axis=0))**2)

def cdnv_tot(features, classes):
  cdnvs = []
  for c1 in range(1,11):
    for c2 in range(c1+1, 11):
      Q1 = features[np.where(classes==c1)[0], :]
      Q2 = features[np.where(classes==c2)[0], :]
      if Q1.shape[0] == 0 or Q2.shape[0] == 0:
        continue
      cdnvs.append(cdnv(Q1, Q2))
  return sum(cdnvs)/len(cdnvs)
  
def get_evaluations(outs_layers, labels):
  cols = []
  vals = []
  labels = labels.detach().cpu().numpy()
  for layer in outs_layers:
    cols.append(f'acc_{layer}')
    features = outs_layers[layer].detach().cpu().numpy()
    features = features.reshape(features.shape[0], -1)
    vals.append(ncc_accuracy(features, labels))
    #cols.append(f'cdnv_{layer}')
    #vals.append(cdnv_tot(features, labels))
  return cols, vals

# Training code

In [15]:
from tqdm import tqdm
from pathlib import Path
import copy
import pandas as pd
import os

In [28]:
def train(model, device, optimizer, scheduler, criterion, dataloaders, savedir, num_epochs, save_interval):
  best_loss = np.inf
  losses = []
  model.to(device)
  for epoch in range(num_epochs):
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()

      running_loss = 0.0
      running_acc = 0.0
      num_preds = 0
      bar = tqdm(dataloaders[phase], desc='Epoch {} {}'.format(epoch, phase).ljust(20))

      for i, batch in enumerate(bar):
        X, y = batch
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
          outs, preds = model(X)
          loss = criterion(outs, y)
          acc = torch.sum(preds == y) / preds.shape[0]
          if phase == 'train':
            loss.backward()
            optimizer.step()

      scheduler.step()
      running_loss += loss.item()
      running_acc += acc.item()
      num_preds += 128
      if i % 10 == 0:
        bar.set_postfix(loss='{:.2f}'.format(running_loss / num_preds))

      epoch_loss = running_loss / num_preds
      epoch_acc = running_acc / num_preds
      # deep copy the model
      if phase == 'val':
        if epoch_loss < best_loss:
          best_loss = epoch_loss
          best_model_wts = copy.deepcopy(model.state_dict())
      outs_layers = model.get_layer_info(X)
      cols, vals = get_evaluations(outs_layers, y)
      df = pd.DataFrame(columns=['epoch', 'phase', 'final_acc']+cols)
      df.loc[0] = [epoch, phase, epoch_acc] + vals
      losses.append(df)
      if (epoch + 1) % save_interval == 0:
        checkpoint = { 
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_sched': scheduler}
        Path(os.path.join(savedir, f'checkpoint_epoch_{epoch+1}.pth')).parent.mkdir(parents=True, exist_ok=True)
        torch.save(checkpoint, os.path.join(os.path.join(savedir, f'checkpoint_epoch_{epoch+1}.pth')))
        losses_int = pd.concat(losses, axis=0, ignore_index=True)
        losses_int.to_csv(os.path.join(savedir, f'losses_epoch_{epoch+1}.csv'), index=False)

  model.load_state_dict(best_model_wts)
  model.eval()

  # Save model weights
  Path(os.path.join(savedir, 'model.pth')).parent.mkdir(parents=True, exist_ok=True)
  torch.save(model.state_dict(), os.path.join(savedir, 'model.pth'))

  losses = pd.concat(losses, axis=0, ignore_index=True)
  losses.to_csv(os.path.join(savedir, 'losses.csv'), index=False)

  return model

# Set up model parameters

In [68]:
n_layers = 8
etf = True
model = CONV(n_layers, 400, etf)
device = torch.device("cuda:0")
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.1)
criterion = torch.nn.CrossEntropyLoss()
savedir = './trial0'
dataloaders = {
    'train': trainloader_10,
    'val': valloader_10
}
save_interval = 1
num_epochs = 1

train(model, device, optimizer, scheduler, criterion, dataloaders, savedir, num_epochs, save_interval)

Epoch 0 val         : 100%|██████████| 79/79 [00:06<00:00, 13.15it/s]


CONV(
  (relu): ReLU()
  (conv_in): ModuleList(
    (0): Conv2d(3, 400, kernel_size=(2, 2), stride=(2, 2))
    (1): Conv2d(400, 400, kernel_size=(2, 2), stride=(2, 2))
  )
  (bn_in): ModuleList(
    (0): BatchNorm2d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm2d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc_out): Linear(in_features=25600, out_features=10, bias=True)
  (classify): Linear(in_features=10, out_features=10, bias=True)
  (convs): ModuleList(
    (0): Conv2d(400, 400, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(400, 400, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(400, 400, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Conv2d(400, 400, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Conv2d(400, 400, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Conv2d(400, 400, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (